In [1]:
import ipynb
from ipynb.fs.full.A_libs_26sep import *
from ipynb.fs.full.B_funs_26sep import *

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [2]:
torch.__version__

'1.6.0'

### Configuration

In [3]:
output_dir = create_output_dir()
seed_everything(seed=42)

Directory  ../results/_231_blending_5dic__DESPUES_NO_SMOOTH  Created 


In [4]:
class CFG:
    num_folds = 7

### DataBases

In [5]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug = pd.read_csv('../input/train_drug.csv')

drug_dict = dict(zip(train_drug['sig_id'],train_drug['drug_id']))

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

train_orig = train_features.merge(train_targets_scored, on='sig_id', how='left')
train_orig = train_orig.merge(train_targets_nonscored, on='sig_id', how='left')
train_orig = train_orig.merge(train_drug, on='sig_id', how='left')

train_noctl = train_orig[train_orig['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)
train_ctl = train_orig[train_orig['cp_type']=='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)

test_noctl = test_features[test_features['cp_type']!='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)
test_ctl = test_features[test_features['cp_type']=='ctl_vehicle'].drop('cp_type', axis=1).reset_index(drop=True)

target = train_noctl[train_targets_scored.columns]

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
target_cols = train_targets_scored.drop('sig_id', axis=1).columns.values.tolist()
target_nonscored_cols = train_targets_nonscored.drop('sig_id', axis=1).columns.values.tolist()
other_feats = []


# mskf = MultilabelStratifiedKFold(n_splits=CFG.num_folds, shuffle=True, random_state=43)
# for f, (t_idx, v_idx) in enumerate(mskf.split(X=train_noctl, y=target)):
#     folds.loc[v_idx, 'kfold'] = int(f)

# folds['kfold'] = folds['kfold'].astype(int)
sum_ones = train_noctl[target_cols].sum(axis=0)

print(train_orig.shape)
print(target.shape)
print(sample_submission.shape)

(23814, 1485)
(21948, 207)
(3982, 207)


### Create Folds based on Drugs and randomsampling

In [6]:
def calc_folds(seed_fold):
    seed_everything(seed=seed_fold)
    # LOCATE DRUGS
    vc = train_noctl.drug_id.value_counts()
    vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
    vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()

#     vc1 = vc.loc[vc <= 18].index.sort_values()
#     vc2 = vc.loc[vc > 18].index.sort_values()

     # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits = CFG.num_folds, shuffle = True, random_state = seed_fold)
    # skf = KFold(n_splits = CFG.num_folds, shuffle = True, random_state = 42)
    tmp = train_noctl.groupby('drug_id')[target_cols].mean().loc[vc1]

    for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits = CFG.num_folds, shuffle = True, random_state = seed_fold)
    # skf = KFold(n_splits = CFG.num_folds, shuffle = True, random_state = 42)
    tmp = train_noctl.loc[train_noctl['drug_id'].isin(vc2)].reset_index(drop = True)
    for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)
    
    # ASSIGN FOLDS
    train_noctl['kfold'] = train_noctl.drug_id.map(dct1)
    train_noctl.loc[train_noctl['kfold'].isna(),'kfold'] = train_noctl.loc[train_noctl['kfold'].isna(),'sig_id'].map(dct2)
    train_noctl['kfold']  = train_noctl['kfold'].astype('int')
    return train_noctl['kfold'].values     

### Blending

In [7]:
from sklearn.metrics import roc_auc_score
def mean_auc(y_true, y_pred):
    auc_mean = []
    for ncol in range(y_true.shape[1]):
        if np.sum(y_true[:,ncol])>0:
            auc_mean.append(roc_auc_score(y_true[:,ncol], y_pred[:, ncol]))
    return np.mean(auc_mean)

In [8]:
# 83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_RANDOM
# 83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics



# 73003_model_TabNet_21nov_4por4_newcalcfold_metrics_RANDOM
# 73002_model_TabNet_21nov_4por4_newcalcfold_metrics


In [9]:
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibratedClassifierCV
names_models = [
#                 '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_73002_model_TabNet_21nov_4por4_newcalcfold_metrics/',
#                 '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_resnet_poly/',
                
               '../results/_FINAL_ANN_CPS_STRAT_KMEANS7_SMOOTH_NO/',
               '../results/_FINAL_TABNET_CPS_STRAT_KMEANS7_SMOOTH_NO/',
               '../results/_FINAL_RESNET_CPS_STRAT_KMEANS7_SMOOTH_NO/']
               
#                '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_RANDOM/',
#                '../results/_NEW_73002_model_TabNet_21nov_4por4_newcalcfold_metrics_RANDOM/',
#                '../results/_NEW_83005_20nov_difffolds_1E3_4por4_newcalcfold_metrics_resnet_poly_RANDOM_v2/']
                
#                '../results/_93005_24nov_original_features/',
#                '../results/_NEW_83007_25nov_NODE/']
folds_cp_list = []
test_noctl_list = []
y_true_list = []
y_pred_list = []
feature_cols_list = []
target_cols_list = []
       
for nmodel, dir_model in enumerate(names_models):
    dir_res = dir_model 
    name_file = [i for i in os.listdir(dir_res) if ('dataframe' in i)][0]
    print(dir_model + name_file)

    with open(dir_res + name_file, 'rb') as file:
        folds_cp= pickle.load(file)
        test_noctl_cp= pickle.load(file)
        y_true_acum= pickle.load(file)
        y_pred_acum= pickle.load(file)
        feature_cols= pickle.load(file)
        target_cols= pickle.load(file)
        losses_df= pickle.load(file)
        scores_seed_df= pickle.load(file)

    folds_cp_list.append(folds_cp)
    test_noctl_list.append(test_noctl_cp)
    y_true_list.append(y_true_acum)
    y_pred_list.append(y_pred_acum)
    target_cols_list.append(target_cols)
    feature_cols_list.append(feature_cols)

    score_test_acum = log_loss_multi(y_true_acum, y_pred_acum)
    auc_test_acum = mean_auc(y_true_acum, y_pred_acum)
    print(score_test_acum, auc_test_acum)
    
    if nmodel>0:
        print(np.sum(y_true_list[0]!=y_true_list[nmodel]))
        print(np.sum(feature_cols_list[0]!=feature_cols_list[nmodel]))
        print(np.sum(target_cols_list[0]!=target_cols_list[nmodel]))       

../results/_FINAL_ANN_CPS_STRAT_KMEANS7_SMOOTH_NO/ann_dataframes.pkl
0.016574918513736298 0.6394355085572886
../results/_FINAL_TABNET_CPS_STRAT_KMEANS7_SMOOTH_NO/tabnet_dataframes.pkl
0.01658246410105539 0.6305546929248357
0
0
0
../results/_FINAL_RESNET_CPS_STRAT_KMEANS7_SMOOTH_NO/ann_dataframes.pkl
0.016579766145847333 0.640391081812387
0
0
0


In [10]:
y_pred_list_orig = y_pred_list.copy()
test_noctl_list_orig = test_noctl_list.copy()

In [11]:
from scipy.optimize import minimize
def combi(list_preds, w_vect, funcion):
    y_blend = []
    for nw, w in enumerate(w_vect):
        y_blend.append(list_preds[nw]*w)
    if funcion=='mean':
        y_blend = np.sum(y_blend, axis=0)
    elif funcion=='harmonic':
        for i in range(len(y_blend)):
            y_blend[i] = 1.0 / y_blend[i] 
        y_blend = len(y_blend)/np.sum(y_blend, axis=0) 
    elif funcion=='geometric':
        y_blend = np.sqrt(np.prod(y_blend, axis=0))
    else:
        print(f'Function {funcion} does not exist!')
        return -1
    return y_blend.clip(0.0, 1.0)
    
def blending_models(w_vect=np.array([1.0, 1.0, 1.0, 1.0]), funcion='mean'):
    y_blend = combi(y_pred_list, w_vect, funcion)
    return log_loss_multi(y_true_list[0], y_blend.clip(0.0, 1.0))

def blending_models_aucmean(w_vect=np.array([1.0, 1.0, 1.0, 1.0]), funcion='mean'):
    y_blend = combi(y_pred_list, w_vect, funcion)
    return -mean_auc(y_true_list[0], y_blend.clip(0.0, 1.0))

In [12]:
# STRATIFIED BLENDING!!!!
# Optimization terminated successfully.
#          Current function value: 0.016590
#          Iterations: 317
#          Function evaluations: 543
# 0.016590380904245444 [0.0921359  0.18177003 0.14607115 0.34632593 0.16019635 0.0780598 ]

In [13]:
# Seleccionamos el metodo
# NEW
n = 3
print("STRATIFIED BLENDING!!!!")
y_pred_list = y_pred_list_orig[:n]
test_noctl_list = test_noctl_list_orig[:n]

metodo = 'mean'
res = minimize(blending_models, np.array([1/n]*n), method='nelder-mead',
                   options={'xatol': 1e-8, 'disp': True}, args=(metodo))

score = res.fun
pesos = res.x
print(score, pesos)

# Combinamos modelos
test_preds = []
for df in test_noctl_list:
    test_preds.append(df[target_cols_list[0]].values)

funcion = metodo
val_values = combi(y_pred_list, pesos, funcion)
test_values = combi(test_preds, pesos, funcion)

blend_test = pd.DataFrame(test_values)
blend_test.columns = target_cols_list[0]
blend_test['sig_id'] = test_noctl_list[0]['sig_id'].values

display(pd.DataFrame([val_values.mean(axis=0), test_values.mean(axis=0)]))

submission = sample_submission.drop(columns=target_cols_list[0])\
.merge(blend_test[['sig_id']+target_cols_list[0]], on='sig_id', how='left')\
.fillna(0.0).reset_index(drop=True)
# sub.to_csv('submission.csv', index=False)
name_sub = output_dir / f'submission_STRAT_{score:.6f}.csv'
submission.to_csv(name_sub, index=False)

print(name_sub)
display(submission.head())
submission_stratified = submission.copy()

STRATIFIED BLENDING!!!!
Optimization terminated successfully.
         Current function value: 0.016476
         Iterations: 113
         Function evaluations: 228
0.016476294101737145 [0.37846299 0.37115626 0.25789995]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
0,0.000976,0.000984,0.001275,0.008979,0.014221,0.003602,0.002588,0.004315,0.000708,0.011667,0.016732,0.002839,0.000526,0.001946,0.000780,0.000787,0.002379,0.004238,0.003854,0.001810,0.001865,0.003471,0.000508,0.002095,0.000788,0.000795,0.000755,0.001048,0.003565,0.001815,0.001265,0.002259,0.002327,0.000536,0.000301,0.000718,0.004141,0.000982,0.003938,0.000504,0.002918,0.003683,0.001836,0.009140,0.004371,0.005465,0.000543,0.001442,0.001679,0.003265,0.001296,0.003122,0.001370,0.000516,0.012486,0.002088,0.002707,0.001861,0.001792,0.001011,0.000764,0.004796,0.001038,0.015265,0.002641,0.001170,0.002163,0.002326,0.002693,0.000510,0.0018,0.019897,0.004754,0.001810,0.001314,0.000508,0.002355,0.017453,0.005810,0.019318,0.014712,0.000517,0.000309,0.007116,0.002242,0.001683,0.000936,0.001303,0.002213,0.011981,0.000903,0.001035,0.001239,0.005033,0.007687,0.002553,0.012451,0.000811,0.003663,0.017179,0.001066,0.002715,0.003372,0.004808,0.002914,0.011396,0.001070,0.001570,0.003360,0.012193,0.004411,0.001722,0.001473,0.001636,0.003376,0.001555,0.002444,0.002015,0.003822,0.011364,0.000506,0.000504,0.002891,0.000777,0.002879,0.000519,0.001414,0.003290,0.003577,0.001325,0.000756,0.004165,0.000981,0.005767,0.002441,0.001881,0.038095,0.000511,0.001451,0.000718,0.001428,0.000559,0.000932,0.003068,0.004652,0.001897,0.002855,0.001267,0.002649,0.012272,0.000999,0.011804,0.001338,0.006366,0.001518,0.002722,0.004768,0.004863,0.001534,0.005034,0.000988,0.001802,0.004044,0.033888,0.002340,0.000506,0.004387,0.001059,0.002846,0.010057,0.000772,0.002882,0.000508,0.001573,0.002026,0.001287,0.010798,0.018624,0.002183,0.001848,0.001854,0.001314,0.012375,0.00136,0.003112,0.000517,0.001007,0.002906,0.001735,0.001095,0.00191,0.001545,0.000547,0.001785,0.005602,0.001031,0.000513,0.001278,0.002362,0.014510,0.003228,0.000500,0.006751,0.001392,0.001684,0.001508
1,0.001023,0.001015,0.001279,0.009189,0.014638,0.003712,0.002674,0.004353,0.000875,0.012044,0.017267,0.002345,0.000532,0.001849,0.000793,0.000801,0.002427,0.004343,0.003949,0.001851,0.001910,0.003457,0.000517,0.002104,0.000811,0.000806,0.000764,0.001039,0.003659,0.001874,0.001299,0.002236,0.002405,0.000545,0.000305,0.000634,0.004396,0.001171,0.003567,0.000516,0.002973,0.003605,0.001918,0.009435,0.004520,0.005593,0.000510,0.001390,0.001569,0.003390,0.001321,0.002931,0.001636,0.000524,0.012598,0.002164,0.002805,0.001922,0.001781,0.001034,0.000777,0.004857,0.001066,0.009350,0.002682,0.002131,0.002192,0.002366,0.002749,0.000521,0.0017,0.020378,0.004707,0.001566,0.001347,0.000521,0.002329,0.017258,0.005925,0.019832,0.014616,0.000525,0.000314,0.007231,0.002236,0.001699,0.000868,0.001352,0.002483,0.010057,0.000776,0.001063,0.001235,0.005080,0.007928,0.002360,0.010924,0.000823,0.003773,0.017642,0.001092,0.003095,0.003380,0.004578,0.002986,0.011771,0.001069,0.001581,0.003467,0.005938,0.003079,0.001762,0.001299,0.001675,0.003238,0.001592,0.002812,0.002042,0.001979,0.010005,0.000516,0.000515,0.002941,0.000789,0.003106,0.000513,0.001503,0.001336,0.003598,0.001338,0.000772,0.004274,0.000965,0.007493,0.002488,0.001946,0.043402,0.000522,0.001499,0.000700,0.001476,0.000576,0.000915,0.003122,0.004750,0.001956,0.002175,0.001321,0.002647,0.011035,0.001020,0.012233,0.001364,0.007709,0.001512,0.002789,0.004855,0.005442,0.001615,0.

../results/_231_blending_5dic__DESPUES_NO_SMOOTH/submission_STRAT_0.016476.csv


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [14]:
# STRATIFIED BLENDING!!!!
# Optimization terminated successfully.
#          Current function value: 0.016597
#          Iterations: 124
#          Function evaluations: 234
# 0.016597055918474674 [0.45949989 0.3209986  0.22388427]

In [15]:
blending_models([0.1075, 0.2098, 0.2107, 0.3352, 0.1414])

IndexError: list index out of range

In [ ]:
# Seleccionamos el metodo
# RANDOM
print("RANDOM BLENDING!!!!")
y_pred_list = y_pred_list_orig[n:]
test_noctl_list = test_noctl_list_orig[n:]
score = res.fun
pesos = res.x
print(score, pesos)

# Combinamos modelos
test_preds = []
for df in test_noctl_list:
    test_preds.append(df[target_cols_list[0]].values)

funcion = metodo
val_values = combi(y_pred_list, pesos, funcion)
test_values = combi(test_preds, pesos, funcion)

blend_test = pd.DataFrame(test_values)
blend_test.columns = target_cols_list[0]
blend_test['sig_id'] = test_noctl_list[0]['sig_id'].values

display(pd.DataFrame([val_values.mean(axis=0), test_values.mean(axis=0)]))

submission = sample_submission.drop(columns=target_cols_list[0])\
.merge(blend_test[['sig_id']+target_cols_list[0]], on='sig_id', how='left')\
.fillna(0.0).reset_index(drop=True)
# sub.to_csv('submission.csv', index=False)
name_sub = output_dir / f'submission_RANDOM_{score:.6f}.csv'
submission.to_csv(name_sub, index=False)

print(name_sub)
display(submission.head())
submission_random= submission.copy()

In [12]:
# from scipy.optimize import minimize
# n = 2
# for metodo in ['mean','harmonic','geometric']:
#     res = minimize(blending_models_aucmean, np.array([1/n]*n), method='nelder-mead',
#                    options={'xatol': 1e-8, 'disp': True}, args=(metodo))
#     print(f'n={n} metodo={metodo} mlogloss={res.fun}, W={res.x}')

In [13]:
from scipy.optimize import minimize
n = len(names_models)
# for metodo in ['mean']: #,'harmonic','geometric']:
#     res = minimize(blending_models, np.array([1/n]*n), method='nelder-mead',
#                    options={'xatol': 1e-8, 'disp': True}, args=(metodo))
#     print(f'n={n} metodo={metodo} mlogloss={res.fun}, W={res.x}')

In [14]:
# Seleccionamos el metodo
metodo = 'mean'
res = minimize(blending_models, np.array([1/n]*n), method='nelder-mead',
                   options={'xatol': 1e-8, 'disp': True}, args=(metodo))

score = res.fun
pesos = res.x
print(score, pesos)

KeyboardInterrupt: 

In [ ]:
         Current function value: 0.016602
         Iterations: 100
         Function evaluations: 205
0.016602136014725388 [0.38940193 0.32797063 0.2861724 ]

In [24]:
blending_models([0.389,0.328,0.286])

0.016602141087921556

In [25]:
# Combinamos modelos
test_preds = []
for df in test_noctl_list:
    test_preds.append(df[target_cols_list[0]].values)

funcion = metodo
val_values = combi(y_pred_list, pesos, funcion)
test_values = combi(test_preds, pesos, funcion)

blend_test = pd.DataFrame(test_values)
blend_test.columns = target_cols_list[0]
blend_test['sig_id'] = test_noctl_list[0]['sig_id'].values

display(pd.DataFrame([val_values.mean(axis=0), test_values.mean(axis=0)]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
0,0.001439,0.001484,0.001685,0.009368,0.014622,0.00399,0.003033,0.004906,0.001158,0.012176,0.017303,0.003233,0.000975,0.002448,0.001204,0.001205,0.002829,0.004615,0.004250,0.002247,0.002303,0.003933,0.000955,0.002475,0.001233,0.001245,0.001239,0.001466,0.004039,0.002283,0.001694,0.002641,0.002741,0.000943,0.000741,0.001127,0.004629,0.001418,0.004304,0.000961,0.003308,0.004133,0.002267,0.009517,0.004778,0.005746,0.000969,0.001826,0.002062,0.003538,0.001718,0.003543,0.001780,0.000959,0.012870,0.002536,0.003108,0.002303,0.002231,0.001467,0.001216,0.005146,0.001450,0.015614,0.003078,0.001561,0.002524,0.002769,0.003070,0.000944,0.002219,0.019977,0.005133,0.002205,0.001774,0.000957,0.002722,0.017513,0.006116,0.019177,0.014848,0.000938,0.000754,0.007463,0.002644,0.002174,0.001342,0.001737,0.002619,0.012094,0.001305,0.001491,0.001655,0.005441,0.008038,0.002993,0.012957,0.001245,0.004006,0.017406,0.001502,0.003093,0.003782,0.005189,0.003383,0.01176,0.001548,0.002028,0.003808,0.012542,0.004781,0.002060,0.001874,0.002042,0.003779,0.001974,0.002864,0.002480,0.004136,0.011576,0.000945,0.000946,0.003323,0.001231,0.003260,0.000961,0.001827,0.003696,0.003970,0.001754,0.001211,0.004636,0.001423,0.006120,0.002922,0.002270,0.038385,0.000949,0.001876,0.001128,0.001831,0.001015,0.001340,0.003458,0.004917,0.002331,0.003258,0.001709,0.003098,0.012432,0.001446,0.012090,0.001810,0.00670,0.001955,0.003119,0.005228,0.005283,0.001983,0.005437,0.001442,0.002290,0.004492,0.034264,0.002748,0.000951,0.004661,0.001478,0.003188,0.010387,0.001208,0.003262,0.000956,0.002088,0.002371,0.001750,0.011267,0.018885,0.002635,0.002258,0.002241,0.001730,0.012885,0.001782,0.003500,0.000945,0.001471,0.003290,0.002163,0.001552,0.002289,0.002033,0.000991,0.002254,0.005921,0.001447,0.000946,0.001690,0.002780,0.015015,0.003578,0.000949,0.007014,0.001837,0.002107,0.001986
1,0.001482,0.001515,0.001688,0.009579,0.015046,0.00409,0.003118,0.004941,0.001296,0.012446,0.017829,0.002719,0.000981,0.002305,0.001217,0.001219,0.002876,0.004717,0.004345,0.002285,0.002345,0.003926,0.000965,0.002482,0.001257,0.001260,0.001246,0.001459,0.004127,0.002341,0.001727,0.002623,0.002814,0.000950,0.000747,0.001048,0.004937,0.001613,0.003984,0.000974,0.003350,0.004048,0.002339,0.009788,0.004917,0.005870,0.000942,0.001791,0.001916,0.003647,0.001741,0.003352,0.002017,0.000967,0.012987,0.002607,0.003203,0.002362,0.002214,0.001490,0.001229,0.005200,0.001478,0.009705,0.003117,0.002491,0.002547,0.002810,0.003116,0.000955,0.002143,0.020498,0.005086,0.001988,0.001805,0.000971,0.002697,0.017290,0.006228,0.019704,0.014719,0.000947,0.000760,0.007565,0.002643,0.002196,0.001298,0.001785,0.002907,0.010228,0.001202,0.001520,0.001652,0.005488,0.008256,0.002814,0.011440,0.001253,0.004111,0.017857,0.001525,0.003493,0.003795,0.004950,0.003457,0.01210,0.001537,0.002030,0.003901,0.006277,0.003474,0.002075,0.001687,0.002078,0.003650,0.002013,0.003205,0.002509,0.002319,0.010244,0.000956,0.000958,0.003377,0.001241,0.003512,0.000949,0.001915,0.001737,0.003991,0.001763,0.001230,0.004742,0.001412,0.007823,0.002971,0.002331,0.043777,0.000962,0.001922,0.001116,0.001872,0.001035,0.001345,0.003506,0.005003,0.002383,0.002522,0.001761,0.003082,0.011271,0.001461,0.012497,0.001835,0.00791,0.001947,0.003185,0.005312,0.005785,0.002071,0.

In [26]:
submission = sample_submission.drop(columns=target_cols_list[0])\
.merge(blend_test[['sig_id']+target_cols_list[0]], on='sig_id', how='left')\
.fillna(0.0).reset_index(drop=True)
# sub.to_csv('submission.csv', index=False)
name_sub = output_dir / f'submission{score:.6f}.csv'
submission.to_csv(name_sub, index=False)
print(name_sub)
display(submission.head())

../results/_229_blending_26nov/submission0.016602.csv


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [27]:
pd.DataFrame(test_values).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
count,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000
mean,0.001482,0.001515,0.001688,0.009579,0.015046,0.004090,0.003118,0.004941,0.001296,0.012446,0.017829,0.002719,0.000981,0.002305,0.001217,0.001219,0.002876,0.004717,0.004345,0.002285,0.002345,0.003926,0.000965,0.002482,0.001257,0.001260,0.001246,0.001459,0.004127,0.002341,0.001727,0.002623,0.002814,0.000950,0.000747,0.001048,0.004937,0.001613,0.003984,0.000974,0.003350,0.004048,0.002339,0.009788,0.004917,0.005870,0.000942,0.001791,0.001916,0.003647,0.001741,0.003352,0.002017,0.000967,0.012987,0.002607,0.003203,0.002362,0.002214,0.001490,0.001229,0.005200,0.001478,0.009705,0.003117,0.002491,0.002547,0.002810,0.003116,0.000955,0.002143,0.020498,0.005086,0.001988,0.001805,0.000971,0.002697,0.017290,0.006228,0.019704,0.014719,0.000947,0.000760,0.007565,0.002643,0.002196,0.001298,0.001785,0.002907,0

In [28]:
pd.DataFrame(val_values).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
count,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000,21948.000000
mean,0.001439,0.001484,0.001685,0.009368,0.014622,0.003990,0.003033,0.004906,0.001158,0.012176,0.017303,0.003233,0.000975,0.002448,0.001204,0.001205,0.002829,0.004615,0.004250,0.002247,0.002303,0.003933,0.000955,0.002475,0.001233,0.001245,0.001239,0.001466,0.004039,0.002283,0.001694,0.002641,0.002741,0.000943,0.000741,0.001127,0.004629,0.001418,0.004304,0.000961,0.003308,0.004133,0.002267,0.009517,0.004778,0.005746,0.000969,0.001826,0.002062,0.003538,0.001718,0.003543,0.001780,0.000959,0.012870,0.002536,0.003108,0.002303,0.002231,0.001467,0.001216,0.005146,0.001450,0.015614,0.003078,0.001561,0.

In [283]:
# from scipy.optimize import minimize
# def combi(list_preds, w_vect, funcion):
#     y_blend = []
#     w_vect = np.concatenate([w_vect, np.array([(1+len(w_vect))-np.sum(w_vect)])])
#     w_vect = w_vect/len(w_vect)
#     for nw, w in enumerate(w_vect):
#         y_blend.append(list_preds[nw]*w)
#     if funcion=='mean':
#         y_blend = np.sum(y_blend, axis=0)
#     elif funcion=='harmonic':
#         for i in range(len(y_blend)):
#             y_blend[i] = 1.0 / y_blend[i] 
#         y_blend = len(y_blend)/np.sum(y_blend, axis=0) 
#     elif funcion=='geometric':
#         y_blend = np.sqrt(np.prod(y_blend, axis=0))
#     else:
#         print(f'Function {funcion} does not exist!')
#         return -1
#     return y_blend #.clip(0.0001, 0.9999)
    
# def blending_models(w_vect=np.array([1.0, 1.0, 1.0, 1.0]), funcion='mean'):
#     y_blend = combi(y_pred_list, w_vect, funcion)
#     return log_loss_multi(y_true_list[0], y_blend.clip(0.0, 1.0))

# def blending_models_aucmean(w_vect=np.array([1.0, 1.0, 1.0, 1.0]), funcion='mean'):
#     y_blend = combi(y_pred_list, w_vect, funcion)
#     return -mean_auc(y_true_list[0], y_blend.clip(0.0, 1.0))

In [323]:
# from scipy.optimize import minimize
# n = 2
# for metodo in ['mean','harmonic','geometric']:
#     res = minimize(blending_models, np.array([1/n]*(n-1)), method='nelder-mead',
#                    options={'xatol': 1e-8, 'disp': True}, args=(metodo))
#     print(f'n={n} metodo={metodo} mlogloss={res.fun}, W={res.x}')

In [270]:
# from scipy.optimize import minimize
# n = 2
# for metodo in ['mean','harmonic','geometric']:
#     res = minimize(blending_models, np.array([1/n]*(n-1)), method='nelder-mead',
#                    options={'xatol': 1e-8, 'disp': True}, args=(metodo))
#     print(f'n={n} metodo={metodo} mlogloss={res.fun}, W={res.x}')

In [271]:
# n = len(names_models)
# res = minimize(blending_models, np.array([1/n]*n), method='nelder-mead',
#                        options={'xatol': 1e-8, 'disp': True}, args=('mean'))

In [324]:
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibratedClassifierCV
names_models = ['../results/_1011_model_ANN_10Folds_KRIDGE_SUBMISSION/',
                '../results/_1126_model_TabNet_10folds_2nov_KRIDGE_SUBMISSION/']
seeds_models = [5, 4]
folds_cp_list = []
test_noctl_list = []
y_true_list = []
y_pred_list = []
feature_cols = []
target_cols_list = []

for nmodel, dir_model in enumerate(names_models):
    print(dir_model)
    dir_res = dir_model #'../results/' + dir_model + '/'
    name_files = [i for i in os.listdir(dir_res) if ('dataframe' in i) and (f'seed{seeds_models[nmodel]}' in i)]
    for name_file in name_files:
        print(name_file)

        with open(dir_res + name_file, 'rb') as file:
            folds_cp= pickle.load(file)
            test_noctl_cp= pickle.load(file)
            y_true= pickle.load(file)
            y_pred= pickle.load(file)
            feature_cols= pickle.load(file)
            target_cols= pickle.load(file)
            losses_list= pickle.load(file)
            scores_seed= pickle.load(file)

            folds_cp_acum= pickle.load(file)
            test_noctl_cp_acum= pickle.load(file)
            y_true_acum= pickle.load(file)
            y_pred_acum= pickle.load(file)
        
        feature_cols.append(feature_cols)
        target_cols_list.append(target_cols)
        test_noctl_list.append(test_noctl_cp_acum)
        y_true_list.append(y_true_test)
        y_pred_list.append(y_pred_test_acum)

        print(log_loss_multi(y_true_acum, y_pred_acum))
        
 


../results/_1011_model_ANN_10Folds_KRIDGE_SUBMISSION/
ann_seed5_dataframes.pkl
0.015791432974833357
../results/_1126_model_TabNet_10folds_2nov_KRIDGE_SUBMISSION/
tabnet_seed4_dataframes.pkl
0.016154480275607952


In [325]:
# score = res.fun
# pesos = res.x
# print(score, pesos)

pesos = np.array([0.96815928, 0.0132194])
score = 0.015695

test_preds = []
for df in test_noctl_list:
    test_preds.append(df[target_cols_list[0]].values)

funcion = 'mean'
val_values = combi(y_pred_list, pesos, funcion)
test_values = combi(test_preds, pesos, funcion)
display(pd.DataFrame([val_values.mean(axis=0), test_values.mean(axis=0)]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
0,0.001413,0.001477,0.001593,0.008815,0.014287,0.004092,0.002835,0.004537,0.001428,0.012491,0.015650,0.003301,0.000990,0.002591,0.001156,0.001166,0.002612,0.004255,0.004221,0.002319,0.002095,0.003948,0.000936,0.002456,0.001179,0.001248,0.001218,0.001451,0.003812,0.002112,0.001749,0.002563,0.002695,0.000950,0.000780,0.001244,0.005803,0.001556,0.004953,0.000900,0.003033,0.003881,0.002165,0.009055,0.004419,0.005756,0.000967,0.001854,0.002098,0.003478,0.001723,0.003739,0.001858,0.000993,0.012659,0.002552,0.002984,0.002269,0.002164,0.001355,0.001176,0.005005,0.001420,0.014520,0.002944,0.001703,0.002509,0.002728,0.002970,0.000962,0.002240,0.018622,0.004841,0.002043,0.001702,0.000997,0.002694,0.016443,0.005764,0.019622,0.013757,0.000936,0.000759,0.007177,0.002686,0.002012,0.001382,0.001779,0.003002,0.012395,0.001431,0.001485,0.001539,0.005327,0.007763,0.002598,0.014265,0.001258,0.003801,0.016262,0.001498,0.003502,0.003628,0.005349,0.003246,0.011064,0.001673,0.002071,0.003456,0.011987,0.004078,0.002368,0.001836,0.001916,0.003874,0.001944,0.003159,0.002606,0.004845,0.012335,0.000914,0.000958,0.003126,0.001196,0.003254,0.000914,0.002066,0.003528,0.003927,0.001666,0.001163,0.004371,0.001406,0.004745,0.002722,0.002234,0.038044,0.000918,0.001873,0.001207,0.001849,0.001030,0.001479,0.003324,0.004817,0.002073,0.003060,0.001559,0.003199,0.013421,0.001513,0.012659,0.001816,0.005977,0.001933,0.003169,0.004639,0.006130,0.002041,0.005217,0.001437,0.002212,0.004072,0.033687,0.00281,0.001013,0.005352,0.001443,0.003136,0.011060,0.001162,0.003097,0.000939,0.002187,0.002029,0.001650,0.010785,0.017667,0.002680,0.002355,0.002107,0.001710,0.012097,0.001771,0.003111,0.000938,0.001518,0.003025,0.001824,0.001455,0.002028,0.001891,0.000938,0.002195,0.006193,0.001472,0.000947,0.001782,0.002550,0.015305,0.003697,0.000913,0.007647,0.001804,0.002144,0.001879
1,0.001490,0.001470,0.001683,0.010030,0.016026,0.004277,0.003198,0.005066,0.001253,0.012019,0.018782,0.002180,0.001048,0.002290,0.001184,0.001190,0.002903,0.004888,0.004594,0.002330,0.002323,0.003976,0.000904,0.002516,0.001232,0.001217,0.001176,0.001404,0.004304,0.002395,0.001726,0.002707,0.002856,0.000922,0.000684,0.000957,0.005019,0.001754,0.003817,0.000912,0.003472,0.004071,0.002410,0.010051,0.005084,0.006193,0.000899,0.001759,0.001699,0.003858,0.001750,0.003339,0.002131,0.000907,0.013426,0.002679,0.003314,0.002405,0.002219,0.001490,0.001175,0.005394,0.001468,0.009061,0.003197,0.003082,0.002610,0.002868,0.003214,0.000905,0.002078,0.020991,0.005196,0.001786,0.001817,0.000908,0.002754,0.016752,0.006511,0.019769,0.014321,0.000913,0.000689,0.007657,0.002680,0.002194,0.001114,0.001789,0.002692,0.009355,0.001081,0.001497,0.001658,0.005614,0.008685,0.002855,0.011601,0.001222,0.004367,0.018993,0.001508,0.003434,0.003872,0.004871,0.003558,0.012597,0.001425,0.002069,0.004019,0.005421,0.003263,0.002114,0.001635,0.002112,0.003684,0.002032,0.003339,0.002514,0.001887,0.008896,0.000885,0.000893,0.003490,0.001176,0.003873,0.000893,0.001531,0.001430,0.004133,0.001794,0.001176,0.004938,0.001457,0.008326,0.003044,0.002372,0.043040,0.000901,0.001947,0.001096,0.001917,0.000973,0.001282,0.003571,0.005219,0.002439,0.002376,0.001716,0.002972,0.009970,0.001426,0.012847,0.001826,0.007697,0.001972,0.003305,0.005492,0.005543,0.0021

In [326]:
# Set columns with low number of ones to 1/sum(ones)
# num_ones = 6
# sum_ones = y_true_list[0].sum(axis=0)
# menor_igual = np.where(sum_ones <= num_ones)[0]
# print(menor_igual)
# for colpos in menor_igual:
#     test_values[:, colpos] = sum_ones[colpos]/len(y_true_list[0])

blend_test = pd.DataFrame(test_values)
blend_test.columns = target_cols_list[0]
blend_test['sig_id'] = test_noctl_list[0]['sig_id'].values

### Create Submission

In [327]:
submission = sample_submission.drop(columns=target_cols_list[0])\
.merge(blend_test[['sig_id']+target_cols_list[0]], on='sig_id', how='left')\
.fillna(0.0).reset_index(drop=True)
# sub.to_csv('submission.csv', index=False)
name_sub = output_dir / f'submission{score:.6f}.csv'
submission.to_csv(name_sub, index=False)
print(name_sub)

../results/_204_blending_03nov_manual/submission0.015695.csv


In [328]:
submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [329]:
submission.describe()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,re

In [330]:
pd.DataFrame(test_values).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
count,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000
mean,0.001490,0.001470,0.001683,0.010030,0.016026,0.004277,0.003198,0.005066,0.001253,0.012019,0.018782,0.002180,0.001048,0.002290,0.001184,0.001190,0.002903,0.004888,0.004594,0.002330,0.002323,0.003976,0.000904,0.002516,0.001232,0.001217,0.001176,0.001404,0.004304,0.002395,0.001726,0.002707,0.002856,0.000922,0.000684,0.000957,0.005019,0.001754,0.003817,0.000912,0.003472,0.004071,0.002410,0.010051,0.005084,0.006193,0.000899,0.001759,0.001699,0.003858,0.001750,0.003339,0.002131,0.000907,0.013426,0.002679,0.003314,0.002405,0.002219,0.001490,0.001175,0.005394,0.001468,0.009061,0.003197,0.003082,0.002610,0.002868,0.003214,0.000905,0.002078,0.020991,0.005196,0.001786,0.001817,0.000908,0.002754,0.016752,0.006511,0.019769,0.014321,0.000913,0.000689,0.007657,0.002680,0.002194,0.001114,0.001789,0.002692,0

In [331]:
pd.DataFrame(val_values).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
count,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000,2195.000000
mean,0.001413,0.001477,0.001593,0.008815,0.014287,0.004092,0.002835,0.004537,0.001428,0.012491,0.015650,0.003301,0.000990,0.002591,0.001156,0.001166,0.002612,0.004255,0.004221,0.002319,0.002095,0.003948,0.000936,0.002456,0.001179,0.001248,0.001218,0.001451,0.003812,0.002112,0.001749,0.002563,0.002695,0.000950,0.000780,0.001244,0.005803,0.001556,0.004953,0.000900,0.003033,0.003881,0.002165,0.009055,0.004419,0.005756,0.000967,0.001854,0.002098,0.003478,0.001723,0.003739,0.001858,0.000993,0.012659,0.002552,0.002984,0.002269,0.002164,0.001355,0.001176,0.005005,0.001420,0.014520,0.002944,0.001703,0.002509,0.002728,0.002970,0.000962,0.002240,0.018622,0.004841,0.002043,0.001702,0.000997,0.002694,0.016443,0.005764,0.019622,0.013757,0.000936,0.000759,0.007177,0.002686,0.002012,0.001382,0.001779,0.003002,0